In [5]:
# !pip install yt_dlp

In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML
import yt_dlp
import os
from ipywidgets import Layout
import requests
import json
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def get_youtube_id(url):
    query = urlparse(url)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    return None

def download_video(url):
    video_id = get_youtube_id(url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")
    
    ydl_opts = {
        'format': 'bestvideo[ext=webm]+bestaudio[ext=webm]/best[ext=webm]',
        'outtmpl': f'{video_id}.%(ext)s'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return f'{video_id}.webm'

def process_with_deepgram(file_path):
    load_dotenv()
    
    url = "https://api.deepgram.com/v1/listen"
    params = {
        "diarize": "true",
        "punctuate": "true",
        "utterances": "true"
    }
    
    DEEPGRAM_API_KEY = os.getenv("DEEPGRAM_API_KEY")
    headers = {
        "Authorization": f"Token {DEEPGRAM_API_KEY}",
        "Content-Type": "video/webm"
    }
    
    with open(file_path, "rb") as file:
        data = file.read()
    
    # Create a retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS", "POST"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    
    # Create a session with the retry strategy
    session = requests.Session()
    session.mount("https://", adapter)
    
    try:
        response = session.post(url, params=params, headers=headers, data=data, verify=True, timeout=30)
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        
        if response.status_code == 200:
            result = response.json()
            output_filename = f"{os.path.splitext(file_path)[0]}_transcript.json"
            with open(output_filename, "w") as outfile:
                json.dump(result, outfile, indent=2)
            print(f"Response saved to {output_filename}")
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request: {e}")
    except json.JSONDecodeError:
        print("Error decoding JSON response")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Create widgets
url_input = widgets.Text(description="YouTube URL:", layout=Layout(width='500px'))
submit_button = widgets.Button(description="Submit")
output = widgets.Output()

# Define button click event
def on_button_click(b):
    with output:
        output.clear_output()
        print("Downloading video...")
        try:
            video_path = download_video(url_input.value)
            print("Download complete!")
            display(HTML(f'<video width="320" height="240" controls><source src="{video_path}" type="video/webm"></video>'))
            
            # Create download link
            download_link = f'<a href="{video_path}" download>Download Video</a>'
            display(HTML(download_link))
            
            print("Processing with Deepgram...")
            process_with_deepgram(video_path)
        except Exception as e:
            print(f"An error occurred: {str(e)}")

# Connect the button click event to the function
submit_button.on_click(on_button_click)

# Display widgets
display(url_input, submit_button, output)

Text(value='', description='YouTube URL:', layout=Layout(width='500px'))

Button(description='Submit', style=ButtonStyle())

Output()

In [ ]:
https://www.youtube.com/watch?v=yJzjyYL8l5Y